# Sentinel-1 backscatter ratio between no-snow (July-September) and wet snow (April, May, June) on an interactive map

inspired by:
https://github.com/gee-community/ee-jupyter-contrib

In [ ]:
# If you get an error here you probably need to authenticate:
#https://github.com/google/earthengine-api/blob/master/python/examples/ipynb/authorize_notebook_server.ipynb
import ipyleaflet
import ee
ee.Initialize()

## Single image from collection

In [ ]:
#https://developers.google.com/earth-engine/sentinel1
s1 = ee.ImageCollection('COPERNICUS/S1_GRD')

In [ ]:
# Filter by area
site = ee.Geometry.Polygon(
    [[-107.65, 37.85],
     [-107.65, 37.95],
     [-107.80, 37.95],
     [-107.80, 37.85],
     [-107.65, 37.85]])
#print(site.toGeoJSON())

In [ ]:
def GetTileLayerUrl(ee_image_object):
  map_id = ee.Image(ee_image_object).getMapId()
  tile_url_template = "https://earthengine.googleapis.com/map/{mapid}/{{z}}/{{x}}/{{y}}?token={token}"
  return tile_url_template.format(**map_id)

In [ ]:
def ReplaceOverlayLayers(map_object, ee_image_object):
    ''' Will update existing map '''
    for lyr in map_object.layers[1:]:
        map_object.remove_layer(lyr)
    tile_url = GetTileLayerUrl(ee_image_object)
    map_object.add_layer(ipyleaflet.TileLayer(url=tile_url))

In [ ]:
# More sophisticated filters
orbitDir = ee.Filter.eq('orbitProperties_pass', 'ASCENDING')
polarization = ee.Filter.listContains('transmitterReceiverPolarisation', 'VV')
mode = ee.Filter.eq('instrumentMode', 'IW')
orbit = ee.Filter.eq('relativeOrbitNumber_start', 49)
resolution = ee.Filter.eq('resolution_meters', 10)
dates = ee.Filter.date('2016-10-01', '2017-09-30')

collection = s1.filter(orbitDir).filter(mode).filter(polarization).select('VV').filterBounds(site)
sample_image = collection.first()

In [ ]:
image_centroid = tuple(site.centroid().getInfo()['coordinates'][::-1])

map1 = ipyleaflet.Map(zoom=12, center=image_centroid, layout=dict(height='600px',width='800px'))
dc = ipyleaflet.DrawControl()
map1.add_control(dc)

viz_params = {'bands':'VV', 
              'min': -25, 
              'max': 0, 
              'opacity': 0.5}

ee_image_object = sample_image.visualize(**viz_params)

map1.add_layer(
    ipyleaflet.TileLayer(url=GetTileLayerUrl(ee_image_object))
)

map1

In [ ]:
# Get difference between snow-free and snow-melting dates (no longer apply region filter)
start = '2017-07-01'
end = '2017-09-30'
snow_free = (
    s1.filterDate(start, end).filter(mode).filter(polarization).select('VV')
        .median()
)

ReplaceOverlayLayers(map1, snow_free.visualize(**viz_params))


In [ ]:
# Amplitude during major snow-melt
start = '2017-05-01'
end = '2017-05-30'
snow_melt= (
    s1.filterDate(start, end).filter(mode).filter(polarization).select('VV')
        .median()       
)

ReplaceOverlayLayers(map1, snow_melt.visualize(**viz_params))

In [ ]:
# Careful w/ client versus server operations
# https://gis.stackexchange.com/questions/247707/retrieve-image-statistics-as-numbers-in-google-earth-engine
dif = snow_melt.subtract(snow_free)

# Use the reducer to get the mean and SD of the image.
#mean = ee.Number(dif.reduceRegion(ee.Reducer.mean(), crs='EPSG:32618', scale=100, geometry=site, bestEffort=True))
#std = ee.Number(dif.reduceRegion(ee.Reducer.stdDev(), crs='EPSG:32618', scale=100, geometry=site, bestEffort=True))
#print(mean.getInfo(), std.getInfo())
#print(mean.getInfo(), std.getInfo())

In [ ]:
viz_dif = {'bands':'VV', 
            'min': -4, 
            'max': 0, 
            'opacity': 0.5,
            'palette': ['0000FF','00FFFF'],}

ReplaceOverlayLayers(map1, dif.visualize(**viz_dif))

In [ ]:
mask = dif.lte(-4)
test = dif.updateMask(mask)
ReplaceOverlayLayers(map1, test.visualize(**viz_dif))

In [ ]:
# Correlate with Elevation / Slope... etc...
#image = ee.Image('USGS/NED')